In [440]:
import os
import numpy as np
from keras.models import Sequential
from keras.optimizers import Adagrad
from keras import regularizers
from keras.layers import Dense, Activation, Dropout
from keras.layers.convolutional import Conv3D
%matplotlib inline


In [441]:
import importlib
import DataLoader
importlib.reload(DataLoader)
from DataLoader import DataGenerator

window_size = 21
training_generator = DataGenerator("../data/train", window_size, 32).generate()
test_generator = DataGenerator("../data/test", window_size, 32).generate()

In [442]:
# Sample some statistics over the training set
total_1s = 0
total_0s = 0

for i in range(100):
    sample_batch_in, sample_batch_out = next(training_generator)
    total_1s += np.sum(sample_batch_out == 1)
    total_0s += np.sum(sample_batch_out == 0)
print("Total 1s: %d, total 0s: %d" % (total_1s, total_0s))

Total 1s: 3158, total 0s: 3242


In [443]:
model = Sequential()
model.add(
    Dense(units=8, 
          activation='relu', 
          input_dim=2*(window_size**3), 
          activity_regularizer=regularizers.l1(10e-5)))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))

# For a binary classification problem
model.compile(optimizer=Adagrad(lr=0.01, epsilon=1e-08, decay=0.0),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model, iterating on the data in batches of 32 samples
model.fit_generator(generator=training_generator,
                    steps_per_epoch = 2000,
                    epochs=100,
                    validation_data = test_generator, 
                    validation_steps = 1000,
                   workers=10)

Epoch 1/100
 992/2000 [=============>................] - ETA: 6s - loss: 0.3624 - acc: 0.8424

In [411]:
import VoxelDistanceFunctionFileLoader
importlib.reload(VoxelDistanceFunctionFileLoader)
from VoxelDistanceFunctionFileLoader import VoxelDistanceFunction

vdf = VoxelDistanceFunction("../data/raw_vdfs/010.vdf")

print(vdf.occupancy.shape, vdf.known.shape, window_size)

center = np.array([50, 30, 70])
width = np.array([20, 20, 20])
step = 1

x_range = range(center[0]-width[0], center[0]+width[0], step)
y_range = range(center[1]-width[1], center[1]+width[1], step)
z_range = range(center[2]-width[2], center[2]+width[2], step)

example_volume = np.zeros((int(np.prod(2*width/step)), 2*(window_size**3)))
k = 0
for x in x_range:
    for y in y_range:
        for z in z_range:
            example_volume[k, :], Y = vdf.GeneratePointwiseSampleAtPoint(window_size, x, y, z)
            k += 1
example_volume_raw, trash = vdf.GenerateLocalRegionAtPoint(width[0]*2, center[0], center[1], center[2], step)

(138, 119, 100) (138, 119, 100) 21


In [412]:
example_predictions = model.predict_on_batch(example_volume)
np.set_printoptions(suppress=True)
print(example_predictions)

[[ 0.00005946]
 [ 0.00003262]
 [ 0.00003118]
 ..., 
 [ 0.00025628]
 [ 0.00072679]
 [ 0.0011021 ]]


In [413]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import ipyvolume
import scipy
def f(x):
    if x == 1:
        v = np.reshape(example_predictions, (2*width/step).astype(int))
    else:
        v = example_volume_raw
    return ipyvolume.quickvolshow(v, lighting=False, width=800, height=400, data_min=np.nanmin(v), data_max=np.nanmax(v))
interact(f, x=widgets.IntSlider(min=0,max=1,step=1,value=0))

A Jupyter Widget

<function __main__.f>

In [414]:
conv_weights = model.layers[0].get_weights()

print(conv_weights[0].shape, conv_weights[1].shape)

def f(x):
    occupancy_weights = np.reshape(conv_weights[0][:window_size**3, x], (window_size, window_size, window_size))
    known_weights = np.reshape(conv_weights[0][window_size**3:, x], (window_size, window_size, window_size))
    bias = conv_weights[1][x]
    print("Bias: ", bias)
    v = occupancy_weights
    return ipyvolume.quickvolshow(v, lighting=False, width=800, height=400, data_min=np.nanmin(v), data_max=np.nanmax(v))
interact(f, x=widgets.IntSlider(min=0,max=conv_weights[1].shape[0],step=1, value=0))


(18522, 8) (8,)


A Jupyter Widget

<function __main__.f>

In [439]:
import tensorflow as tf

from keras import backend as K
tf_session = K.get_session()

inp = tf.Variable(np.random.rand(1, 2*(window_size**3)).astype(np.float32),name="inp")
tf_session.run(tf.variables_initializer([inp], name='init'))

optim = tf.train.GradientDescentOptimizer(0.5).minimize(-model(inp))
tf_session.run(optim, feed_dict={K.learning_phase(): 0})
v = tf_session.run(inp)
v = v[0, :window_size**3]
v = np.reshape(v, (window_size, window_size, window_size))
ipyvolume.quickvolshow(v, lighting=False, width=800, height=400, data_min=np.nanmin(v), data_max=np.nanmax(v))

A Jupyter Widget